In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
.appName('DAG and Broadcast')\
.getOrCreate()

In [2]:
rides_data = [
("R001","U001","Hyderabad",12.5,240,"Completed"),
("R002","U002","Delhi",8.2,180,"Completed"),
("R003","U003","Mumbai",15.0,300,"Cancelled"),
("R004","U004","Bangalore",5.5,120,"Completed"),
("R005","U005","Hyderabad",20.0,360,"Completed"),
("R006","U006","Delhi",25.0,420,"Completed"),
("R007","U007","Mumbai",7.5,150,"Completed"),
("R008","U008","Bangalore",18.0,330,"Completed"),
("R009","U009","Delhi",6.0,140,"Cancelled"),
("R010","U010","Hyderabad",10.0,200,"Completed")
]
rides_cols = [
"ride_id",
"user_id",
"city",
"distance_km",
"duration_seconds",
"status"
]

rides_df = spark.createDataFrame(rides_data, rides_cols)

In [3]:
surge_data = [
("Hyderabad",1.2),
("Delhi",1.5),
("Mumbai",1.8),
("Bangalore",1.3)
]
surge_cols = ["city","surge_multiplier"]
surge_df = spark.createDataFrame(surge_data, surge_cols)

#EXERCISE SET 1 — TRANSFORMATIONS vs ACTIONS

Exercise 1.1

Create a transformation pipeline that:
Filters only Completed rides
Selects ride_id , city , distance_km

Tasks:
Do not trigger any action

Explain whether Spark executed anything: no it did not execute anything

In [4]:
completed_rides_df = (
    rides_df
    .filter("status = 'Completed'")
    .select("ride_id", "city", "distance_km")
)

Exercise 1.2

Trigger a single action on the pipeline. Tasks:

Identify which line caused execution: .show() Explain why previous lines did not execute. it does not execute till an action is triggered.

In [5]:
completed_rides_df.show()

+-------+---------+-----------+
|ride_id|     city|distance_km|
+-------+---------+-----------+
|   R001|Hyderabad|       12.5|
|   R002|    Delhi|        8.2|
|   R004|Bangalore|        5.5|
|   R005|Hyderabad|       20.0|
|   R006|    Delhi|       25.0|
|   R007|   Mumbai|        7.5|
|   R008|Bangalore|       18.0|
|   R010|Hyderabad|       10.0|
+-------+---------+-----------+



#EXERCISE SET 2 — DAG & LINEAGE

Exercise 2.1

Create a transformation chain with:
Multiple filters
A column selection
Tasks:
Run explain(True)
Identify:
Logical plan
Optimized logical plan
Physical plan

In [7]:
df_chain = (
    rides_df
    .filter("status = 'Completed'")
    .filter("distance_km > 10")
    .select("ride_id", "city", "distance_km")
)
df_chain.explain(True)

== Parsed Logical Plan ==
'Project ['ride_id, 'city, 'distance_km]
+- Filter (distance_km#3 > cast(10 as double))
   +- Filter (status#5 = Completed)
      +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Analyzed Logical Plan ==
ride_id: string, city: string, distance_km: double
Project [ride_id#0, city#2, distance_km#3]
+- Filter (distance_km#3 > cast(10 as double))
   +- Filter (status#5 = Completed)
      +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Optimized Logical Plan ==
Project [ride_id#0, city#2, distance_km#3]
+- Filter ((isnotnull(status#5) AND isnotnull(distance_km#3)) AND ((status#5 = Completed) AND (distance_km#3 > 10.0)))
   +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Physical Plan ==
*(1) Project [ride_id#0, city#2, distance_km#3]
+- *(1) Filter ((isnotnull(status#5) AND isnotnull(distance_km#3)) AND ((s

Exercise 2.2

Reorder transformations (filter after join vs before join).
Tasks:
Compare DAGs
Identify which plan is more efficient and why

In [8]:
#bad plan
bad_plan = rides_df.join(surge_df, "city").filter("distance_km > 10")
bad_plan.explain(True)

== Parsed Logical Plan ==
'Filter ('distance_km > 10)
+- Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
   +- Join Inner, (city#2 = city#6)
      :- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
      +- LogicalRDD [city#6, surge_multiplier#7], false

== Analyzed Logical Plan ==
city: string, ride_id: string, user_id: string, distance_km: double, duration_seconds: bigint, status: string, surge_multiplier: double
Filter (distance_km#3 > cast(10 as double))
+- Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
   +- Join Inner, (city#2 = city#6)
      :- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
      +- LogicalRDD [city#6, surge_multiplier#7], false

== Optimized Logical Plan ==
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multipli

In [9]:
#good plan
good_plan = rides_df.filter("distance_km > 10").join(surge_df, "city")
good_plan.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [city])
:- Filter (distance_km#3 > cast(10 as double))
:  +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
+- LogicalRDD [city#6, surge_multiplier#7], false

== Analyzed Logical Plan ==
city: string, ride_id: string, user_id: string, distance_km: double, duration_seconds: bigint, status: string, surge_multiplier: double
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- Filter (distance_km#3 > cast(10 as double))
   :  +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
   +- LogicalRDD [city#6, surge_multiplier#7], false

== Optimized Logical Plan ==
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- Filter ((isnotnull(distance_km#3) AND (distance_km#3 > 10.0))

filter before join is better as it reduces data before shuffle, smaller join input, less network io, fewer cpu cycles

#EXERCISE SET 3 — PARTITIONS & SHUFFLE

Exercise 3.1

Check the number of partitions of rides_df .
Tasks:
Repartition into 4 partitions

Coalesce into 1 partition
Observe number of output files when writing to Parquet

In [10]:
rides_df.rdd.getNumPartitions()

2

In [11]:
rides_4 = rides_df.repartition(4)
rides_4.rdd.getNumPartitions()

4

In [12]:
rides_1 = rides_df.coalesce(1)
rides_1.rdd.getNumPartitions()

1

In [13]:
rides_4.write.mode("overwrite").parquet("out_4")
rides_1.write.mode("overwrite").parquet("out_1")

Exercise 3.2

Repartition rides by city .
Tasks:
Run explain(True)
Identify whether a shuffle is introduced

In [14]:
rides_city = rides_df.repartition("city")
rides_city.explain(True)

== Parsed Logical Plan ==
'RepartitionByExpression ['city]
+- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Analyzed Logical Plan ==
ride_id: string, user_id: string, city: string, distance_km: double, duration_seconds: bigint, status: string
RepartitionByExpression [city#2]
+- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Optimized Logical Plan ==
RepartitionByExpression [city#2]
+- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange hashpartitioning(city#2, 200), REPARTITION_BY_COL, [plan_id=182]
   +- Scan ExistingRDD[ride_id#0,user_id#1,city#2,distance_km#3,duration_seconds#4L,status#5]



Shuffle is introduced as data redistributed based on city, requires exchange

#EXERCISE SET 4 — JOIN WITHOUT BROADCAST (BAD DAG)

Exercise 4.1

Join rides_df with surge_df on city without using broadcast.
Tasks:
Run explain(True)
Identify:
Join type
Exchange operators
Sort operations
Stage boundaries

In [15]:
join_df = rides_df.join(surge_df, "city")
join_df.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [city])
:- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
+- LogicalRDD [city#6, surge_multiplier#7], false

== Analyzed Logical Plan ==
city: string, ride_id: string, user_id: string, distance_km: double, duration_seconds: bigint, status: string, surge_multiplier: double
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
   +- LogicalRDD [city#6, surge_multiplier#7], false

== Optimized Logical Plan ==
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- Filter isnotnull(city#2)
   :  +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
   +- Filter isnotnull(city#6)
    

Exercise 4.2

Apply a filter ( distance_km > 10 ) before the join.
Tasks:
Observe whether shuffle is removed
Explain why or why not

In [16]:
filtered_join = rides_df.filter("distance_km > 10").join(surge_df, "city")
filtered_join.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [city])
:- Filter (distance_km#3 > cast(10 as double))
:  +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
+- LogicalRDD [city#6, surge_multiplier#7], false

== Analyzed Logical Plan ==
city: string, ride_id: string, user_id: string, distance_km: double, duration_seconds: bigint, status: string, surge_multiplier: double
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- Filter (distance_km#3 > cast(10 as double))
   :  +- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
   +- LogicalRDD [city#6, surge_multiplier#7], false

== Optimized Logical Plan ==
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- Filter ((isnotnull(distance_km#3) AND (distance_km#3 > 10.0))

#EXERCISE SET 5 — BROADCAST JOIN (GOOD DAG)

Exercise 5.1

Apply a broadcast hint to surge_df .
Tasks:
Run explain(True)
Identify:
Join type
BroadcastExchange
Disappearance of shuffles

In [17]:
from pyspark.sql.functions import broadcast
broadcast_join = rides_df.join(
    broadcast(surge_df),
    "city"
)
broadcast_join.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [city])
:- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [city#6, surge_multiplier#7], false

== Analyzed Logical Plan ==
city: string, ride_id: string, user_id: string, distance_km: double, duration_seconds: bigint, status: string, surge_multiplier: double
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6)
   :- LogicalRDD [ride_id#0, user_id#1, city#2, distance_km#3, duration_seconds#4L, status#5], false
   +- ResolvedHint (strategy=broadcast)
      +- LogicalRDD [city#6, surge_multiplier#7], false

== Optimized Logical Plan ==
Project [city#2, ride_id#0, user_id#1, distance_km#3, duration_seconds#4L, status#5, surge_multiplier#7]
+- Join Inner, (city#2 = city#6), rightHint=(strategy=broadcast)
   :- Filter isnotnull(city#2)
   :  +- LogicalRDD [rid

Exercise 5.2

Compare physical plans from:
Exercise 4.1
Exercise 5.1
Tasks:
List operators that disappeared
Explain performance impact

Operators that disappeared with Broadcast Join (Exercise 5.1 vs. Exercise 4.1):

When you introduced the broadcast hint in Exercise 5.1, the following operators disappeared from the physical plan:

Sort operations on both rides_df and surge_df: In SortMergeJoin (Exercise 4.1), both DataFrames needed to be sorted by the join key (city) before the merge could happen. The BroadcastHashJoin (Exercise 5.1) eliminates this need.
Exchange hashpartitioning for rides_df: In SortMergeJoin, both DataFrames had to be shuffled (repartitioned) by the join key so that rows with the same key would reside on the same partition across the cluster. With BroadcastHashJoin, only the smaller surge_df is broadcasted, so the larger rides_df does not need to be shuffled.
Performance Impact:

The disappearance of these operators, particularly the Sort and Exchange (shuffles) for the larger rides_df, has a significant positive performance impact:

Reduced Network I/O: Shuffling data across the network is one of the most expensive operations in Spark. BroadcastHashJoin avoids the shuffle of the larger DataFrame entirely, drastically reducing network traffic.
Reduced CPU Overhead: Sorting large datasets is CPU-intensive. BroadcastHashJoin removes this sorting step.
Faster Join Execution: By eliminating these costly operations, BroadcastHashJoin can be orders of magnitude faster than SortMergeJoin when the broadcasted table is small enough to fit comfortably into the memory of each executor. The BroadcastExchange for surge_df is still a network operation, but it's a one-time transfer of a small dataset to all executors, which is much more efficient than repeatedly shuffling a large dataset.


#EXERCISE SET 6 — DAG INTERPRETATION

Exercise 6.1

From the physical plan:
Identify all expensive operators
Classify them as CPU, memory, or network heavy

Exchange hashpartitioning (Shuffle):

Classification: Primarily Network-heavy, also CPU-heavy and can be Memory/Disk I/O-heavy.
Explanation: This operator is responsible for redistributing data across the cluster, typically when data needs to be grouped, joined, or sorted by a key. It involves serializing data, transferring it over the network, and then deserializing it on the receiving nodes. This is often the most expensive operation in Spark due to the high network overhead and potential disk spills if data doesn't fit in memory.
Sort:

Classification: Primarily CPU-heavy and Memory-heavy, potentially Disk I/O-heavy.
Explanation: Sorting data requires significant computational effort. If the data to be sorted exceeds the available memory on an executor, Spark will spill the data to disk, leading to expensive disk I/O.
SortMergeJoin:

Classification: Network-heavy, CPU-heavy, Memory-heavy.
Explanation: This join strategy combines the costs of shuffling (via Exchange hashpartitioning) and sorting (Sort) on both DataFrames before the merge phase. The sorting and merging process itself also consumes CPU and memory.
BroadcastExchange (part of BroadcastHashJoin):

Classification: Primarily Network-heavy and Memory-heavy.
Explanation: While efficient for small tables, broadcasting involves sending the entire smaller DataFrame to all executor nodes. This is a network operation, and each executor must store the broadcasted data in memory.
BroadcastHashJoin:

Classification: CPU-heavy and Memory-heavy (after the BroadcastExchange).
Explanation: After the broadcast, the smaller DataFrame is converted into a hash table in memory on each executor. The larger DataFrame is then scanned, and its keys are used to probe this in-memory hash table. Hash table creation and probing are CPU-intensive and require memory for the hash table itself.
Filter: (Less expensive than shuffles/joins, but can still be significant)

Classification: Primarily CPU-heavy.
Explanation: Applying filter conditions involves iterating through rows and evaluating predicates. For very large datasets or complex conditions, this can consume

Exercise 6.2

Explain why Spark defaults to SortMergeJoin .

Reliability and Scalability: SortMergeJoin is a more robust and generally scalable join algorithm. It works efficiently regardless of the size of the DataFrames being joined. BroadcastHashJoin, while fast for small tables, can fail if the table to be broadcasted is too large to fit into the memory of a single executor.

No Data Size Assumption: Spark's optimizer doesn't always know the exact size of the DataFrames at the planning stage, especially if they are derived from complex transformations. SortMergeJoin doesn't require prior knowledge about data size to guarantee execution, making it a safe default.

Memory Constraints: Broadcating a large table can exhaust the memory of executors, leading to out-of-memory errors. SortMergeJoin handles larger datasets by spilling data to disk if memory is insufficient during the sort phase, which is slower but prevents crashes.

No Broadcast Limit: By default, Spark has a configuration (spark.sql.autoBroadcastJoinThreshold) that sets a size limit for auto-broadcasting. If a DataFrame is smaller than this threshold, Spark might automatically use BroadcastHashJoin even without an explicit hint. However, if it exceeds this threshold (or if the threshold is not met for other reasons), SortMergeJoin is the fallback.

Handling Skewed Data: SortMergeJoin can handle skewed data distributions more gracefully than HashJoin (which BroadcastHashJoin is a variant of). In HashJoin, if a key has many values, the single task processing that key can become a bottleneck. SortMergeJoin distributes the work more evenly after the initial sort.

#EXERCISE SET 7 — ACTION-DRIVEN EXECUTION

Exercise 7.1

Create a long transformation pipeline without any action.
Tasks:
Explain what Spark has done so far

Exercise 7.2

Trigger different actions ( count , show , write ) separately.
Tasks:
Observe whether Spark recomputes the DAG
Explain behavior

Exercise 7.1

Create a long transformation pipeline without any action.
Tasks:
Explain what Spark has done so far

In [18]:
from pyspark.sql.functions import col

long_pipeline_df = (
    rides_df
    .filter(col("status") == "Completed")
    .filter(col("distance_km") > 5)
    .withColumn("duration_hours", col("duration_seconds") / 3600)
    .select("ride_id", "city", "distance_km", "duration_hours")
    .filter(col("duration_hours") < 1)
)

# No action is called here
# long_pipeline_df.show()
# long_pipeline_df.count()
# long_pipeline_df.write.parquet("output")

Exercise 7.2

Trigger different actions ( count , show , write ) separately.
Tasks:
Observe whether Spark recomputes the DAG
Explain behavior

Exercise 7.2

Trigger different actions ( count , show , write ) separately.
Tasks:
Observe whether Spark recomputes the DAG
Explain behavior

In [19]:
# Triggering count() action
print("Count of rows in long_pipeline_df:", long_pipeline_df.count())

Count of rows in long_pipeline_df: 8


In [20]:
# Triggering show() action
print("First 20 rows of long_pipeline_df:")
long_pipeline_df.show()

First 20 rows of long_pipeline_df:
+-------+---------+-----------+--------------------+
|ride_id|     city|distance_km|      duration_hours|
+-------+---------+-----------+--------------------+
|   R001|Hyderabad|       12.5| 0.06666666666666667|
|   R002|    Delhi|        8.2|                0.05|
|   R004|Bangalore|        5.5| 0.03333333333333333|
|   R005|Hyderabad|       20.0|                 0.1|
|   R006|    Delhi|       25.0| 0.11666666666666667|
|   R007|   Mumbai|        7.5|0.041666666666666664|
|   R008|Bangalore|       18.0| 0.09166666666666666|
|   R010|Hyderabad|       10.0| 0.05555555555555555|
+-------+---------+-----------+--------------------+



In [21]:
# Triggering write() action
print("Writing long_pipeline_df to parquet...")
long_pipeline_df.write.mode("overwrite").parquet("output_pipeline")
print("Write complete.")

Writing long_pipeline_df to parquet...
Write complete.


#EXERCISE SET 8 — THINKING QUESTIONS (WRITTEN)

1. Why does broadcast remove shuffle from the DAG?

Answer: Shuffle is a mechanism in Spark to repartition data across the cluster, typically to bring data with the same key together for operations like joins or aggregations. It involves significant network I/O, which is often a bottleneck.
When you use a broadcast join (specifically BroadcastHashJoin), Spark sends the entire smaller DataFrame (or RDD) to all executor nodes in the cluster. This means that every executor now has a local copy of the smaller dataset. When the join operation then occurs, each executor can join its partition of the larger DataFrame with the local copy of the smaller DataFrame. Because the smaller dataset is already present on every node where the larger dataset's partitions reside, there's no need to exchange any data over the network to match keys for the larger DataFrame. This eliminates the need for a shuffle operation on the larger side of the join, making it highly efficient for joins with small lookup tables.

2. Why does repartition always introduce shuffle?

Answer: Repartition is an operation used to change the number of partitions of a DataFrame, or to redistribute data based on certain columns. When you invoke repartition, especially if you specify a different number of partitions or repartition by specific columns, Spark needs to ensure that the data is distributed correctly according to the new partitioning scheme.
To achieve this, data from existing partitions must be moved to potentially new partitions across different executor nodes. For example, if you repartition by a column, all rows with the same value for that column must end up in the same new partition. This process of collecting data from various source partitions and redistributing it to target partitions across the network is inherently a shuffle operation. A shuffle involves serializing data, transferring it over the network, and then deserializing it on the receiving nodes, which is an expensive process.

3. Why is coalesce cheaper than repartition?

Answer: Both coalesce and repartition modify the number of partitions, but they do so differently, impacting their cost.
Repartition can both increase and decrease the number of partitions, and it always performs a full shuffle. This means data from all existing partitions might be redistributed across the cluster to form the new partitions, even if the number of partitions is decreasing.
Coalesce, in contrast, can only decrease the number of partitions. The key difference in its implementation is that it tries to avoid a full shuffle. Instead of redistributing all data, coalesce combines existing partitions on the same worker nodes. For instance, if you have 10 partitions and coalesce to 5, Spark might combine two existing partitions into one new partition on the same machine without moving data across the network. This minimizes data movement and network I/O, making coalesce a significantly cheaper operation than repartition when the goal is solely to reduce the number of partitions.

4. Why does Spark delay execution until an action?

Answer: Spark employs a fundamental concept known as lazy evaluation. This means that when you define transformations (such as filter, select, withColumn, join), Spark does not immediately execute these operations or process any data. Instead, it internally builds an execution plan, represented as a Directed Acyclic Graph (DAG), which outlines the sequence of operations that need to be performed.
The actual computation (data processing) only begins when an action (like count, show, collect, write, saveAsTable) is invoked. Delaying execution until an action offers several significant advantages:
Optimization: By having the complete lineage of transformations available before execution, Spark's Catalyst optimizer can analyze the entire DAG, reorder operations (e.g., push down filters to read less data), combine redundant steps, and apply various sophisticated optimizations to create a much more efficient physical execution plan. This can drastically reduce the amount of data read, shuffled, and processed.
Fault Tolerance: If a node fails during an operation, Spark can intelligently recompute only the lost partitions from the last checkpoint or the original data source using the information in the DAG, rather than having to restart the entire computation from scratch.
Resource Management: Knowing the full scope of work through the DAG allows Spark to better manage and allocate cluster resources (CPU, memory, network) more effectively for the entire job.